# Final Project: CHECKPOINT 1. 
Github link: https://github.com/kashyapkolukula/CMSC320_S25_Project

Datasets: 
* NBA Salaries: Hoops Fortune (2020-2025) https://www.kaggle.com/datasets/omarsobhy14/nba-players-salaries?resource=download
* NBA Stats (1947-present) https://www.kaggle.com/datasets/sumitrodatta/nba-aba-baa-stats




# Final Project: CHECKPOINT 2. 
TODO: Submit a Jupyter notebook file (.ipynb) to Gradescope

Directions:
* (5 points) Data preprocessing: (a) import, (b) parse (e.g., convert strings to ints), (c) organize   (e.g., set up a database or a pandas DataFrame).
* (20 points) Basic data exploration and summary statistics
* You must present three conclusions using at least three different statistical methods, including hypothesis testing.
* For example: What are the main characteristics of your dataset? How many features and entries are there? Is a feature over-represented? Are features correlated? Are there outliers? Identify the attributes that will affect your choice of primary analysis technique. Etcetera.
* For each method, you must have at least one gorgeous plot.




In [ ]:
# downloaded the kaggle zip files